In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.layers import Bidirectional, TimeDistributed,SpatialDropout2D, Conv2D, MaxPool2D, Input, GRU, LSTM,Dense, Activation, Dropout,Flatten, Reshape, Permute
from keras.layers import BatchNormalization,ZeroPadding1D
from keras.models import Sequential,Model
import keras.backend as K
import os
import numpy as np
K.set_image_data_format('channels_last')
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib.pyplot as plt
import librosa.display as dis
import librosa
from keras_preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import gc
gc.collect()

25

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Method1**- By Using spectrogram coefficient as input to the network

In [2]:
df=pd.read_csv("/content/drive/MyDrive/annotations.csv")
X_train=np.zeros((1000,128,2500,1))
Y_train=np.zeros((1000,10))
classes={
    "Bark":0,
    "Meow":1,
    "Siren":2,
    "Shatter":3,
    "Knock":4,
    "Crying_and_sobbing":5,
    "Microwave_oven":6,
    "Vehicle_horn_and_car_horn_and_honking":7,
    "Doorbell":8,
    "Walk_and_footsteps":9
}

Resizing each Spectrogram to (128,2500,1) by Zero padding

In [3]:
i=0
path="/content/drive/MyDrive/train"
for filename in os.listdir(path):
    file=f"{path}/{filename}"
    x=np.load(file)
    #print(filename)
    l=list(df.label[df.fname==filename])
    l=l[0]
    if x.shape[2]<2500:
        X_train[i,:,0:x.shape[2],0]=x[0,:,:]
        X_train[i,:,x.shape[2]:2500,0]=0
    else:
        X_train[i,:,0:2500,0]=x[0,:,0:2500]
    Y_train[i,classes[l]]=1
    i=i+1

In [ ]:
print(Y_train[0])
print(X_train[0].shape)
X_train,Y_train=shuffle(X_train,Y_train)
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.20, random_state=42)

In [5]:
pool=tf.keras.layers.GlobalAveragePooling2D()

Convolutional Neural Network

In [6]:
def build_model():
    model = Sequential()
    model.add(Conv2D(input_shape=(128,2500,1),filters=4,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=4,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(pool)
    model.add(Flatten())
    # model.add(Dense(units=64,activation="relu"))
    model.add(Dense(units=32,activation="relu"))
    model.add(Dense(units=10, activation="softmax"))
    model.compile(optimizer='Adam', loss=keras.losses.categorical_crossentropy, metrics=[keras.metrics.CategoricalAccuracy()])
    model.summary()
    return model

In [ ]:
m=build_model()

In [8]:
def accuracy(x,y,mod):
    y_pred=mod.predict(x)
    y_pred=np.argmax(y_pred,axis=1)
    y_actual=np.argmax(y,axis=1)
    r=0
    for i in range(x.shape[0]):
        if y_pred[i]==y_actual[i]:
            r=r+1
    return r/x.shape[0]

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model1.h5', monitor='val_loss', verbose=1, save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early]
m.fit(x_train,y_train,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list,
                     batch_size=32,
                    epochs=20
)

In [9]:
model_b = keras.models.load_model('/content/drive/MyDrive/best_model.h5')
print(accuracy(x_test,y_test,model_b))

0.93


In [ ]:
model_b.evaluate(
    x=x_test,
    y=y_test,
    batch_size=32,
    verbose='auto',
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=False,
)

In [ ]:
gc.collect()

Preprocessing(Zero padding) test dataset for prediction 

In [10]:
i=0
x_testf=np.zeros((201,128,2500,1))
path="/content/drive/MyDrive/test/Audio_Classification-MLSP-test"
for filename in os.listdir(path):
    file=f"{path}/{filename}"
    x=np.load(file)
    if x.shape[2]<2500:
        x_testf[i,:,0:x.shape[2],0]=x[0,:,:]
        x_testf[i,:,x.shape[2]:2500,0]=0
    else:
        x_testf[i,:,0:2500,0]=x[0,:,0:2500]
    i=i+1
    if i>=201:
        break

In [11]:
y=model_b.predict(x_testf)

In [12]:
y_p=np.argmax(y, axis=1)

In [47]:
key_list = list(classes.keys())
val_list = list(classes.values())
position = [val_list.index(p) for p in y_p]
key=[key_list[p] for p in position]

In [49]:
test_df=pd.DataFrame()
test_df["id"]=os.listdir("/content/drive/MyDrive/test/Audio_Classification-MLSP-test")

In [50]:
test_df["label"]=key

In [51]:
test_df.to_csv("/content/drive/MyDrive/test_p.csv")

**Method 2** By converting spectrogram to images 

In [18]:
path="/content/drive/MyDrive/train"
for filename in os.listdir(path):
  file=f"{path}/{filename}"
  x=np.load(file)
  x=x[0]
  fig = plt.Figure()
  canvas = FigureCanvas(fig)
  ax = fig.add_subplot(111)
  p = dis.specshow(librosa.amplitude_to_db(x, ref=np.max), ax=ax, y_axis='log', x_axis='time')
  f=f"{filename[:-4]}.png"
  fig.savefig(f"/content/train_images/{f}")

Loading image files using opencv

In [ ]:
i=0
gc.collect()
import cv2 as cv
x_trainf=np.zeros((1000,288,432,3))
y_trainf=np.zeros((1000,10))
path1="/content/train_images"
for filename in os.listdir(path1):
    if filename[0]=='.':
      continue
    file=f"{path1}/{filename}"
    x=cv.imread(file)
    x=x/255
    #print(filename)
    filename_npy=f"{filename[:-4]}.npy"
    l=list(df.label[df.fname==filename_npy])
    l=l[0]
    x_trainf[i]=x
    y_trainf[i,classes[l]]=1
    i=i+1

change input shape to (288,432,3) in previous model
(288,432,3) is the shape of all spectrogram images

In [ ]:
def build_model1():
    model = Sequential()
    model.add(Conv2D(input_shape=(288,432,3),filters=4,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=4,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(SpatialDropout2D(rate=0.3))
    model.add(pool)
    model.add(Flatten())
    # model.add(Dense(units=64,activation="relu"))
    model.add(Dense(units=32,activation="relu"))
    model.add(Dense(units=10, activation="softmax"))
    model.compile(optimizer='Adam', loss=keras.losses.categorical_crossentropy, metrics=[keras.metrics.CategoricalAccuracy()])
    model.summary()
    return model

In [ ]:
m1=build_model1()

In [ ]:
x_trainf,y_trainf=shuffle(x_trainf,y_trainf)
x_trainf1, x_testf1, y_trainf1, y_testf1 = train_test_split(x_trainf,y_trainf, test_size=0.20, random_state=42)

In [ ]:
checkpoint = ModelCheckpoint('/content/sample_data/best_modeli.h5', monitor='val_loss', verbose=1, save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=15)
callbacks_list = [checkpoint, early]
m1.fit(x_trainf1,y_trainf1,
                    validation_data=(x_testf1, y_testf1),
                    callbacks=callbacks_list,
                     batch_size=32,
                    epochs=30
)

In [ ]:
m1.evaluate(
    x=x_testf1,
    y=y_testf1,
    batch_size=32,
    verbose='auto',
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,s
    return_dict=False,
)

Method 1 is performing better than method 2